<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br /><b>Peter Br&ouml;cker</b></td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Vorlesung &mdash; Programmiertechniken 12 / Part II
</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2016</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml)

# Part II &mdash; Neuronales Netz mit Stochastic Gradient Descent und Mini Batches

Wir arbeiten nun mit dem sogenannten <a href="https://en.wikipedia.org/wiki/MNIST_database">MNIST Datensatz</a> handschriftlicher Ziffern, der standardmäßig dazu genutzt wird, um die Qualität verschiedener Designs von neuronalen Netzen zu überprüfen. Dazu laden wir ihn zunächst hinunter und untersuchen dann den Inhalt.

In [ ]:
using PyCall
@pyimport matplotlib.cm as cm
using PyPlot
using LaTeXStrings

### Der <a href="https://en.wikipedia.org/wiki/MNIST_database">MNIST Datensatz</a> handschriftlicher Ziffern

In [ ]:
if !isfile("mnist_train.csv")
    download("http://www.thp.uni-koeln.de/trebst/Lectures/CompPhys-2016/mnist_train.csv")
end

if !isfile("mnist_test.csv")
    download("http://www.thp.uni-koeln.de/trebst/Lectures/CompPhys-2016/mnist_test.csv")
end

using DataFrames

test = readtable("mnist_test.csv", header=false);
train = readtable("mnist_train.csv", header=false);        

println("Größe der Datensätze")
println(size(test))
println(size(train))

Die erste Spalte ist das Label, während der Rest eine digitalisierte, handschriftliche Ziffer der Größe $28 \times 28$ darstellt. Wir separieren nun Label von Zahl und betrachten dann ein paar Beispielziffern.

In [ ]:
sze = 28
train_labels = Array(train[1]);
train_labels = zeros(size(train)[1], 10)
for i in 1:size(train)[1]
    train_labels[i, train[i, 1] + 1] = 1
end
train_inputs = Array(train[2:end]) / 255;
test_labels = zeros(size(test)[1], 10)
for i in 1:size(test)[1]
    test_labels[i, test[i, 1] + 1] = 1
end
test_inputs = Array(test[2:end]) / 255;

### Visualisierung

In [ ]:
figure(figsize=(10, 10))

println("Beispiele Training-Set")
subplot(221)
println(train_labels[1, :],"   Ziffer = ", indmax(train_labels[1, :])-1)
imshow(transpose(reshape(train_inputs[1, :, :], sze, sze)), interpolation="none", cmap="gist_gray")
gca()[:xaxis][:set_visible](false)
gca()[:yaxis][:set_visible](false)

subplot(222)
println(train_labels[2, :],"   Ziffer = ", indmax(train_labels[2, :])-1)
imshow(transpose(reshape(train_inputs[2, :, :], sze, sze)), interpolation="none", cmap="gist_gray")
gca()[:xaxis][:set_visible](false)
gca()[:yaxis][:set_visible](false)

println("\nBeispiele Test-Set")
subplot(223)
println(test_labels[5, :],"   Ziffer = ", indmax(test_labels[5, :])-1)
imshow(transpose(reshape(test_inputs[5, :, :], sze, sze)), interpolation="none", cmap="gist_gray")
gca()[:xaxis][:set_visible](false)
gca()[:yaxis][:set_visible](false)

subplot(224)
println(test_labels[6, :],"   Ziffer = ", indmax(test_labels[6, :])-1)
imshow(transpose(reshape(test_inputs[6, :, :], sze, sze)), interpolation="none", cmap="gist_gray")
gca()[:xaxis][:set_visible](false)
gca()[:yaxis][:set_visible](false)


## Mini batches

Anstatt nun alle Datensätze auf einmal zu verwenden, wie wir es in dem ersten, einfacheren Beispiel getan haben, verwenden wir nun einzelne Teilmengen, sogenannte **mini batches**, wobei es wichtig ist, dass diese Teilmengen groß genug sind, um die Gesamtemenge gut genug zu repräsentieren. Es ist wichtig vor jedem Durchlauf die Daten zu mischen, um zu verhindern, dass in den Daten Zyklen enstehen, in denen das Netzwerk hängen bleibt. Die Anzahl der Durchläufe, die mit den gesamten Daten passieren, wird in diesem Kontext als *epoch* bezeichnet. Wie viele man davon braucht, um eine Netzwerk ausreichend gut zu trainineren, hängt sehr stark vom Problem ab und muss empirisch bestimmt werden. 

In [ ]:
sigmoid(x) = 1. ./ (1. + exp(-x))
s_prime(x) = sigmoid(x) .* (1. - sigmoid(x));

**Netzwerk-Architektur**

In [ ]:
n_input = 28 * 28
n_hidden = 150
n_output = 10

H = 2 * (rand(n_input, n_hidden) - 0.5);
HB = 2 * (rand(1, n_hidden) - 0.5);
O = 2 * (rand(n_hidden, n_output) - 0.5);
HO = 2 * (rand(1, n_output) - 0.5);

In [ ]:
println(size(train_inputs[1, :] * H))

**Überwachtes Lernen** des neuronalen Netzes mit **Back Propagation**, **Stochastic Gradient Descent** und **Mini Batches**

In [ ]:
epochs = 1000
gamma = 0.01 
batch_size = 250
println(size(train_inputs))
println(Int(size(train_inputs)[1] / batch_size))
batches = 2 #Int(size(train_inputs)[1] / batch_size)
order = collect(1:size(train_inputs)[1])

hidden = zeros(n_hidden)
result = zeros(n_output)

for epoch in 1:epochs
    if mod(epoch, 10) == 0
        print("Epoch:\t", epoch, " - ")
    end
    shuffle!(order)
    for b in 1:batches-1
        dO = zeros(O)
        dHO = zeros(HO)
        dH = zeros(H)
        dHB = zeros(HB)
        for i in order[b * batch_size:(b + 1) * batch_size - 1]
            input_data = train_inputs[i, :]
            target_data = train_labels[i, :]

            hidden = sigmoid(input_data * H + HB)
            result = sigmoid(hidden * O + HO)
        
            result_err = target_data - result
            result_delta = result_err .* s_prime(result)

            hidden_err = result_delta * O'
            hidden_delta = hidden_err .* s_prime(hidden)

            dO += gamma * hidden' * result_delta
            dHO += gamma * result_delta
            dH += gamma * input_data' * hidden_delta
            dHB += gamma * hidden_delta
        end
        O += dO
        HO += dHO
        H += dH
        HB += dHB
    end
    
    test_error = 0

    for i in size(test_inputs)[1]
        test_error += sum(0.5*(sigmoid(sigmoid(test_inputs[i, :] * H + HB) * O + HO) - test_labels[i, :]).^2)
    end
    
    if mod(epoch, 10) == 0
        println("Error:\t", test_error)
    end
end

**Anwendung** des optimierten Netzes auf den Test-Datensatz

In [ ]:
check = floor(10000*rand())

imshow(transpose(reshape(test_inputs[check, :, :], sze, sze)), interpolation="none", cmap="gist_gray")
gca()[:xaxis][:set_visible](false)
gca()[:yaxis][:set_visible](false)

test_input = test_labels[check, :]
println("Test-Ziffer = ", indmax(test_input)-1)

println("\nAusgabe des neuronalen Netzes")
neuro_result = round(sigmoid(sigmoid(test_inputs[check, :] * H + HB) * O + HO), 3)
println("   ", neuro_result)
println("   Erkannte Ziffer = ", indmax(neuro_result)-1)

Berechnung der **Treffer-/Fehlerquote**

In [ ]:
oops = 0

for i in 1:10000
#     check = floor(10000*rand())
    check = i

    test_input = test_labels[check, :]
    neuro_result = round(sigmoid(sigmoid(test_inputs[check, :] * H + HB) * O + HO), 3)

    input_digit = indmax(test_input)-1
    output_digit = indmax(neuro_result)-1
    
    if input_digit != output_digit
        oops += 1
    end
end

println("Trefferquote ist ", 100-oops/100, " Prozent.")
println("Fehlerquote liegt bei ", oops/100," Prozent.")